In [237]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'ciudad':str, 'provincia': str}

training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','ciudad','idzona','piscina','garages'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','provincia','ciudad','idzona','piscina','garages'])

In [238]:
training_data.piscina.sum()*training_data.garages.mean()

32441.03495438475

In [239]:
cant_zonas = (training_data.groupby('ciudad')['idzona'].unique().to_frame().reset_index())
cant_zonas['zonas_por_ciudad'] = (training_data.groupby('ciudad')['idzona'].unique().to_frame().reset_index())['idzona'].apply(lambda x: ((x.size)-1) if (np.isnan(x).any()) else (x.size))
cant_zonas.drop(columns={'idzona'},inplace=True)
cant_zonas.head(10)

,ciudad,zonas_por_ciudad
0,Abalá,0
1,Abasolo,3
2,Abejones,0
3,Acajete,0
4,Acambay,2
5,Acaponeta,0
6,Acapulco de Juárez,191
7,Acateno,0
8,Acatic,1
9,Acatzingo,2


In [240]:
garages_ciudad = training_data.groupby('ciudad').agg({'garages':'mean','piscina':'sum'}).reset_index().rename(columns={'garages':'mean_gar_ciudad','piscina':'sum_pisc'})
datos_ciudad = pd.merge(cant_zonas, garages_ciudad, on='ciudad', how='right')
#datos_ciudad.info()
mean_gar = datos_ciudad.mean_gar_ciudad.mean()
datos_ciudad.mean_gar_ciudad.fillna(mean_gar,inplace=True)

training_data = pd.merge(training_data,datos_ciudad, on='ciudad', how='left')
training_data.mean_gar_ciudad.fillna(mean_gar,inplace=True)

training_data.head(20)

,id,ciudad,provincia,garages,idzona,piscina,zonas_por_ciudad,mean_gar_ciudad,sum_pisc
0,254099,Benito Juárez,Distrito Federal,1.0,23533.0,0.0,54.0,1.403898,257.0
1,53461,La Magdalena Contreras,Distrito Federal,2.0,24514.0,0.0,42.0,1.839674,35.0
2,247984,Tonalá,Jalisco,2.0,48551.0,0.0,127.0,1.325997,13.0
3,209067,Zinacantepec,Edo. de México,1.0,53666.0,0.0,32.0,1.685039,0.0
4,185997,Zapopan,Jalisco,1.0,47835.0,0.0,480.0,1.652776,1165.0
5,126147,Coyoacán,Distrito Federal,1.0,23650.0,0.0,117.0,1.521178,74.0
6,139233,Oaxaca de Juárez,Oaxaca,1.0,73510.0,0.0,79.0,1.299578,8.0
7,5013,Playa del Carmen,Quintana Roo,2.0,130510.0,0.0,1.0,1.423398,176.0
8,44962,Villa de Alvarez,Colima,1.0,9010.0,0.0,76.0,1.275362,1.0
9,134537,Ixtapaluca,Edo. de México,NaN,59171.0,0.0,52.0,1.213693,7.0


In [241]:
training_data.garages.fillna(training_data.mean_gar_ciudad,inplace=True)
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 9 columns):
id                  240000 non-null int32
ciudad              239628 non-null object
provincia           239845 non-null object
garages             240000 non-null float64
idzona              211379 non-null float64
piscina             240000 non-null float64
zonas_por_ciudad    239628 non-null float64
mean_gar_ciudad     240000 non-null float64
sum_pisc            239628 non-null float64
dtypes: float64(6), int32(1), object(2)
memory usage: 17.4+ MB


In [242]:
training_data.idzona.fillna(-1, inplace=True)
props_zone = (((training_data.idzona.value_counts()>5)).to_frame().reset_index()).rename(columns={'index':'idzona','idzona':'many_props'})

#props_zone.loc[props_zone.idzona==-1] #TENGO QUE CAMBIAR ESTE
props_zone['many_props'].value_counts()

False    13895
True      5825
Name: many_props, dtype: int64

In [243]:
training_data.ciudad.fillna('no_city', inplace=True)

In [244]:
garages_ciudad = training_data.groupby('ciudad').agg({'garages':'mean','piscina':'sum','zonas_por_ciudad':'median'}).reset_index()
garages_ciudad['gar_pisc_ciudad'] = ((garages_ciudad.garages*garages_ciudad.piscina)+1)/(garages_ciudad.zonas_por_ciudad+1)
garages_ciudad.drop(columns={'garages','piscina','zonas_por_ciudad'},inplace=True)
garages_ciudad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 2 columns):
ciudad             876 non-null object
gar_pisc_ciudad    875 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [245]:
training_data = pd.merge(training_data,props_zone, on='idzona', how='left')
training_data = pd.merge(training_data,garages_ciudad, on= 'ciudad', how='left')
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 11 columns):
id                  240000 non-null int32
ciudad              240000 non-null object
provincia           239845 non-null object
garages             240000 non-null float64
idzona              240000 non-null float64
piscina             240000 non-null float64
zonas_por_ciudad    239628 non-null float64
mean_gar_ciudad     240000 non-null float64
sum_pisc            239628 non-null float64
many_props          240000 non-null bool
gar_pisc_ciudad     239628 non-null float64
dtypes: bool(1), float64(7), int32(1), object(2)
memory usage: 19.5+ MB


In [246]:
training_data.tail()

,id,ciudad,provincia,garages,idzona,piscina,zonas_por_ciudad,mean_gar_ciudad,sum_pisc,many_props,gar_pisc_ciudad
239995,119879,Zinacantepec,Edo. de México,2.0,53666.0,0.0,32.0,1.685039,0.0,True,0.030303
239996,259178,Toluca,Edo. de México,3.0,51954.0,0.0,187.0,1.572982,23.0,True,0.197758
239997,131932,Benito Juárez,Distrito Federal,1.0,50003995.0,0.0,54.0,1.403898,257.0,True,6.578215
239998,146867,Iztapalapa,Distrito Federal,0.0,24162.0,0.0,208.0,1.112600,4.0,True,0.026078
239999,121958,Cuajimalpa de Morelos,Distrito Federal,2.0,23750.0,0.0,52.0,1.965517,547.0,True,20.304489


In [247]:
gar_pisc_zona = training_data.groupby('idzona').agg({'piscina':'sum','garages':'mean','many_props':'median'}).reset_index()
gar_pisc_zona['gar_pisc_zona'] = gar_pisc_zona.garages*gar_pisc_zona.piscina
#precio_zona.loc[(~(precio_zona['+5props']))|(precio_zona.idzona==-1), 'precio_por_metro'] = precio_zona.precio_mts_ciudad
gar_pisc_zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 5 columns):
idzona           19720 non-null float64
piscina          19720 non-null float64
garages          19720 non-null float64
many_props       19720 non-null bool
gar_pisc_zona    19720 non-null float64
dtypes: bool(1), float64(4)
memory usage: 635.6 KB


In [248]:
gar_pisc_zona = gar_pisc_zona[['idzona','gar_pisc_zona','many_props']]

In [249]:
gar_pisc_zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 3 columns):
idzona           19720 non-null float64
gar_pisc_zona    19720 non-null float64
many_props       19720 non-null bool
dtypes: bool(1), float64(2)
memory usage: 327.5 KB


In [250]:
###precio_zona.to_csv('../../res/ftr/precio_por_metro_idzona.csv')

In [251]:
training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','idzona','ciudad'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','idzona','ciudad'])

training_data.idzona.fillna(-1, inplace=True)
evaluation_data.idzona.fillna(-1, inplace=True)


In [252]:
evaluation_data=pd.merge(evaluation_data,gar_pisc_zona,on='idzona',how='left')
evaluation_data=pd.merge(evaluation_data,garages_ciudad,on='ciudad',how='left')

evaluation_data.gar_pisc_ciudad.fillna(evaluation_data.gar_pisc_ciudad.mean(),inplace=True)
evaluation_data.gar_pisc_zona.fillna(evaluation_data.gar_pisc_zona,inplace=True)


training_data=pd.merge(training_data,gar_pisc_zona,on='idzona',how='left')
training_data=pd.merge(training_data,garages_ciudad,on='ciudad',how='left')

training_data.gar_pisc_ciudad.fillna(training_data.gar_pisc_ciudad.mean(),inplace=True)
training_data.gar_pisc_zona.fillna(training_data.gar_pisc_zona,inplace=True)

In [253]:
training_data.many_props.fillna(False,inplace=True)
evaluation_data.many_props.fillna(False,inplace=True)
evaluation_data.head(15)

,id,ciudad,idzona,gar_pisc_zona,many_props,gar_pisc_ciudad
0,4941,Miguel Hidalgo,-1.0,2484.186347,True,11.656623
1,51775,Mérida,113851.0,264.755150,True,10.207488
2,115253,Coyoacán,23620.0,0.000000,True,0.962434
3,299321,Acapulco de Juárez,129347.0,22.588235,True,9.910312
4,173570,Tultitlán,57125.0,0.000000,True,0.037636
5,30862,Zapopan,48216.0,0.000000,False,4.005165
6,244471,Monterrey,323485.0,0.000000,True,1.227223
7,127794,Lerma,54688.0,0.000000,True,2.163301
8,71558,Veracruz,107969.0,13.829268,True,0.604260
9,218011,Cancún,50002836.0,NaN,False,10.875951


In [254]:
evaluation_data.loc[((evaluation_data['many_props'])==False)|(evaluation_data.idzona==-1), 'gar_pisc_zona'] = evaluation_data.gar_pisc_ciudad
training_data.loc[((training_data['many_props'])==False)|(training_data.idzona==-1), 'gar_pisc_zona'] = training_data.gar_pisc_ciudad


In [255]:
training_data = training_data[['id','gar_pisc_zona']]
evaluation_data = evaluation_data[['id','gar_pisc_zona']]

In [256]:
evaluation_data.loc[evaluation_data.gar_pisc_zona == 0]
#HAY QUE VER SI PONERLE EL VALOR DE LA CIUDAD A LOS QUE DAN CERO

,id,gar_pisc_zona
2,115253,0.0
4,173570,0.0
6,244471,0.0
7,127794,0.0
14,195272,0.0
15,238663,0.0
17,207082,0.0
19,220946,0.0
24,116904,0.0
25,208352,0.0


In [258]:
training_data.to_csv('../../res/ftr/gar_pisc_zone_train.csv')
evaluation_data.to_csv('../../res/ftr/gar_pisc_zone_evaluation.csv')